In [ ]:
!unzip -q vaegan.zip

In [ ]:
import os.path as osp

In [ ]:
# Set Gogle Drive Connection
if not osp.exists("./drive"):
  from google.colab import drive
  drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
torchvision_version = !python -c "import torchvision; print(torchvision.__version__)"
if torchvision_version[0] != '0.4.0':
  print("Incorrect torchvision version is installed. Uninstalling.")
  # Uninstall old torchvision
  !pip uninstall -y torchvision
  print("Installing torchvision 0.4.0")
  # Install proper torchvision
  !pip install torchvision==0.4.0 --upgrade

Incorrect torchvision version is installed. Uninstalling.
Uninstalling torchvision-0.3.0:
  Successfully uninstalled torchvision-0.3.0
Installing torchvision 0.4.0
     |████████████████████████████████| 8.8MB 2.8MB/s 
     |████████████████████████████████| 748.9MB 18kB/s 
  Found existing installation: torch 1.1.0
    Uninstalling torch-1.1.0:
      Successfully uninstalled torch-1.1.0


In [ ]:
if not osp.exists("apex"):
  print("Cloning NVIDIA APEX")
  !git clone https://github.com/NVIDIA/apex

Cloning NVIDIA APEX
Cloning into 'apex'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 5076 (delta 2), reused 2 (delta 0), pack-reused 5069
Receiving objects: 100% (5076/5076), 8.82 MiB | 5.16 MiB/s, done.
Resolving deltas: 100% (3275/3275), done.


In [ ]:
vis_inst = !python -c 'import pkgutil; print(1 if pkgutil.find_loader("apex") else 0)'
if not int(vis_inst[0]):
  print("Apex not installed")
  !cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
else:
  print("Apex is already installed")

Apex not installed
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:243: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-v6__vp3b
Created temporary directory: /tmp/pip-req-tracker-gtnh1zou
Created requirements tracker '/tmp/pip-req-tracker-gtnh1zou'
Created temporary directory: /tmp/pip-install-0gdwylfb
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-33_wi562
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-gtnh1zou'
    Running setup.py (path:/tmp/pip-req-build-33_wi562/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.2.0
    running egg_info
    creating pip-egg-info/apex.egg-info
    writing pip-egg-info/apex.egg-info/PKG-INFO
    writing dependency_links to pip-egg

In [ ]:
!pip install kaptan==0.5.12

  Created wheel for kaptan: filename=kaptan-0.5.12-cp36-none-any.whl size=10976 sha256=b35a4ae75b21d9377cd486bd396c05e460d2da130ece9919438bf6afd220c0fa
  Stored in directory: /root/.cache/pip/wheels/27/7c/59/f347d4c2e1504da5cd2eb40d07cbcb341aefbdb1682d83a4aa
Successfully built kaptan


In [ ]:
import os, shutil, glob, re, pdb, json
import kaptan
#import click
import utils
import torch, torchsummary, torchvision

In [ ]:
def main(config, mode, weights):
    # Generate configuration
    cfg = kaptan.Kaptan(handler='yaml')
    config = cfg.import_config(config)

    # Generate logger
    MODEL_SAVE_NAME, MODEL_SAVE_FOLDER, LOGGER_SAVE_NAME, CHECKPOINT_DIRECTORY = utils.generate_save_names(config)
    logger = utils.generate_logger(MODEL_SAVE_FOLDER, LOGGER_SAVE_NAME)

    logger.info("*"*40);logger.info("");logger.info("")
    logger.info("Using the following configuration:")
    logger.info(config.export("yaml", indent=4))
    logger.info("");logger.info("");logger.info("*"*40)

    """ SETUP IMPORTS """
    #from crawlers import ReidDataCrawler
    #from generators import SequencedGenerator
    

    #from loss import LossBuilder
    

    NORMALIZATION_MEAN, NORMALIZATION_STD, RANDOM_ERASE_VALUE = utils.fix_generator_arguments(config)
    TRAINDATA_KWARGS = {"rea_value": config.get("TRANSFORMATION.RANDOM_ERASE_VALUE")}


    """ Load previousely saved logger, if it exists """
    DRIVE_BACKUP = config.get("SAVE.DRIVE_BACKUP")
    if DRIVE_BACKUP:
        backup_logger = os.path.join(CHECKPOINT_DIRECTORY, LOGGER_SAVE_NAME)
        if os.path.exists(backup_logger):
            shutil.copy2(backup_logger, ".")
    else:
        backup_logger = None

    NUM_GPUS = torch.cuda.device_count()
    if NUM_GPUS > 1:
        raise RuntimeError("Not built for multi-GPU. Please start with single-GPU.")
    logger.info("Found %i GPUs"%NUM_GPUS)


    # --------------------- BUILD GENERATORS ------------------------
    # Supported integrated data sources --> MNIST, CIFAR
    # For BDD or others need a crawler and stuff...but we;ll deal with it later
    from generators import ClassedGenerator

    load_dataset = config.get("EXECUTION.DATASET_PRELOAD")
    if load_dataset in ["MNIST", "CIFAR10", "CIFAR100"]:
        crawler = load_dataset
        #dataset = torchvision.datasets.MNIST(root="./MNIST", train=True,)
        logger.info("No crawler necessary when using %s dataset"%crawler)
    else:
        raise NotImplementedError()
        # import crawler
        # crawler = crawler...
        # logger.info("Crawling data folder %s"%config.get("DATASET.ROOT_DATA_FOLDER"))
        # crawler = ReidDataCrawler(data_folder = config.get("DATASET.ROOT_DATA_FOLDER"), train_folder=config.get("DATASET.TRAIN_FOLDER"), test_folder = config.get("DATASET.TEST_FOLDER"), query_folder=config.get("DATASET.QUERY_FOLDER"), **{"logger":logger})

    train_generator = ClassedGenerator( gpus=NUM_GPUS, i_shape=config.get("DATASET.SHAPE"), \
                                        normalization_mean=NORMALIZATION_MEAN, normalization_std=NORMALIZATION_STD, normalization_scale=1./config.get("TRANSFORMATION.NORMALIZATION_SCALE"), \
                                        h_flip = config.get("TRANSFORMATION.H_FLIP"), t_crop=config.get("TRANSFORMATION.T_CROP"), rea=config.get("TRANSFORMATION.RANDOM_ERASE"), 
                                        **TRAINDATA_KWARGS)    
    train_generator.setup(  crawler, preload_classes = config.get("EXECUTION.DATASET_PRELOAD_CLASS"), \
                            mode='train',batch_size=config.get("TRANSFORMATION.BATCH_SIZE"), \
                            workers = config.get("TRANSFORMATION.WORKERS"))
    logger.info("Generated training data generator")

    test_generator = ClassedGenerator( gpus=NUM_GPUS, i_shape=config.get("DATASET.SHAPE"), \
                                        normalization_mean=NORMALIZATION_MEAN, normalization_std=NORMALIZATION_STD, normalization_scale=1./config.get("TRANSFORMATION.NORMALIZATION_SCALE"), \
                                        h_flip = config.get("TRANSFORMATION.H_FLIP"), t_crop=config.get("TRANSFORMATION.T_CROP"), rea=config.get("TRANSFORMATION.RANDOM_ERASE"), 
                                        **TRAINDATA_KWARGS)    
    test_generator.setup(  crawler, preload_classes = config.get("EXECUTION.DATASET_TEST_PRELOAD_CLASS"), \
                            mode='test',batch_size=config.get("TRANSFORMATION.BATCH_SIZE"), \
                            workers = config.get("TRANSFORMATION.WORKERS"))
    logger.info("Generated testing data generator")


    # --------------------- INSTANTIATE MODEL ------------------------
    model_builder = __import__("models", fromlist=["*"])
    model_builder = getattr(model_builder, config.get("EXECUTION.MODEL_BUILDER"))
    logger.info("Loaded {} from {} to build VAEGAN model".format(config.get("EXECUTION.MODEL_BUILDER"), "models"))

    vaegan_model = model_builder(   arch=config.get("MODEL.ARCH"), base=config.get("MODEL.BASE"), \
                                    latent_dimensions = config.get("MODEL.LATENT_DIMENSIONS"), \
                                    **json.loads(config.get("MODEL.MODEL_KWARGS")))
    logger.info("Finished instantiating model")

    if mode == "test":
        vaegan_model.load_state_dict(torch.load(weights))
        vaegan_model.cuda()
        vaegan_model.eval()
    else:
        vaegan_model.cuda()
        #logger.info(torchsummary.summary(vaegan_model, input_size=(config.get("TRANSFORMATION.CHANNELS"), *config.get("DATASET.SHAPE"))))
        logger.info(torchsummary.summary(vaegan_model.Encoder, input_size=(config.get("TRANSFORMATION.CHANNELS"), *config.get("DATASET.SHAPE"))))
        logger.info(torchsummary.summary(vaegan_model.Decoder, input_size=(config.get("MODEL.LATENT_DIMENSIONS"), 1)))
        logger.info(torchsummary.summary(vaegan_model.LatentDiscriminator, input_size=(config.get("MODEL.LATENT_DIMENSIONS"), 1)))
        logger.info(torchsummary.summary(vaegan_model.Discriminator, input_size=(config.get("TRANSFORMATION.CHANNELS"), *config.get("DATASET.SHAPE"))))
    

    # --------------------- INSTANTIATE LOSS ------------------------
    # ----------- NOT NEEDED. VAEGAN WILL USE BCE LOSS THROUGHOUT 
    # loss_function = LossBuilder(loss_functions=config.get("LOSS.LOSSES"), loss_lambda=config.get("LOSS.LOSS_LAMBDAS"), loss_kwargs=config.get("LOSS.LOSS_KWARGS"), **{"logger":logger})
    # logger.info("Built loss function")
    # --------------------- INSTANTIATE OPTIMIZER ------------------------
    optimizer_builder = __import__("optimizer", fromlist=["*"])
    optimizer_builder = getattr(optimizer_builder, config.get("EXECUTION.OPTIMIZER_BUILDER"))
    logger.info("Loaded {} from {} to build VAEGAN model".format(config.get("EXECUTION.OPTIMIZER_BUILDER"), "optimizer"))

    OPT = optimizer_builder(base_lr=config.get("OPTIMIZER.BASE_LR"))
    optimizer = OPT.build(vaegan_model, config.get("OPTIMIZER.OPTIMIZER_NAME"), **json.loads(config.get("OPTIMIZER.OPTIMIZER_KWARGS")))
    logger.info("Built optimizer")
    # --------------------- INSTANTIATE SCHEDULER ------------------------
    try:
        scheduler = __import__('torch.optim.lr_scheduler', fromlist=['lr_scheduler'])
        scheduler_ = getattr(scheduler, config.get("SCHEDULER.LR_SCHEDULER"))
    except (ModuleNotFoundError, AttributeError):
        scheduler_ = config.get("SCHEDULER.LR_SCHEDULER")
        scheduler = __import__("scheduler."+scheduler_, fromlist=[scheduler_])
        scheduler_ = getattr(scheduler, scheduler_)
    scheduler = {}
    for base_model in ["Encoder", "Decoder", "Discriminator", "Autoencoder", "LatentDiscriminator"]:
        scheduler[base_model] = scheduler_(optimizer[base_model], last_epoch = -1, **json.loads(config.get("SCHEDULER.LR_KWARGS")))
        logger.info("Built scheduler for {}".format(base_model))
    

    # --------------------- SETUP CONTINUATION  ------------------------
    if DRIVE_BACKUP:
        fl_list = glob.glob(os.path.join(CHECKPOINT_DIRECTORY, "*.pth"))
    else:
        fl_list = glob.glob(os.path.join(MODEL_SAVE_FOLDER, "*.pth"))
    _re = re.compile(r'.*epoch([0-9]+)\.pth')
    previous_stop = [int(item[1]) for item in [_re.search(item) for item in fl_list] if item is not None]
    if len(previous_stop) == 0:
        previous_stop = 0
    else:
        previous_stop = max(previous_stop) + 1
        logger.info("Previous stop detected. Will attempt to resume from epoch %i"%previous_stop)

    # --------------------- INSTANTIATE TRAINER  ------------------------
    Trainer = __import__("trainer", fromlist=["*"])
    Trainer = getattr(Trainer, config.get("EXECUTION.TRAINER"))
    logger.info("Loaded {} from {} to build VAEGAN model".format(config.get("EXECUTION.TRAINER"), "trainer"))

    loss_stepper = Trainer(model=vaegan_model, loss_fn = None, optimizer = optimizer, scheduler = scheduler, train_loader = train_generator.dataloader, test_loader = test_generator.dataloader, epochs = config.get("EXECUTION.EPOCHS"), batch_size = config.get("TRANSFORMATION.BATCH_SIZE"), latent_size = config.get("MODEL.LATENT_DIMENSIONS"), logger = logger)
    loss_stepper.setup(step_verbose = config.get("LOGGING.STEP_VERBOSE"), save_frequency=config.get("SAVE.SAVE_FREQUENCY"), test_frequency = config.get("EXECUTION.TEST_FREQUENCY"), save_directory = MODEL_SAVE_FOLDER, save_backup = DRIVE_BACKUP, backup_directory = CHECKPOINT_DIRECTORY, gpus=NUM_GPUS, fp16 = config.get("OPTIMIZER.FP16"), model_save_name = MODEL_SAVE_NAME, logger_file = LOGGER_SAVE_NAME)
    if mode == 'train':
      loss_stepper.train(continue_epoch=previous_stop)
    elif mode == 'test':
      loss_stepper.evaluate()
    else:
      raise NotImplementedError()

In [ ]:
main("config/vaegan_cifar10.yml", "train","")

05:28:44-130 ****************************************
05:28:44-132 
05:28:44-134 
05:28:44-137 Using the following configuration:
05:28:44-146 DATASET:
    QUERY_FOLDER: ''
    ROOT_DATA_FOLDER: CIFAR10
    SHAPE: [32, 32]
    TEST_FOLDER: ''
    TRAIN_FOLDER: ''
EXECUTION:
    DATASET_PRELOAD: CIFAR10
    DATASET_PRELOAD_CLASS: [0, 1, 2]
    DATASET_TEST_PRELOAD_CLASS: [3, 4, 5, 6, 7, 8, 9]
    EPOCHS: 121
    MODEL_BUILDER: vaegan_model_builder
    MODEL_SERVING: None
    OPTIMIZER_BUILDER: VAEGANOptimizerBuilder
    TEST_FREQUENCY: 5
    TRAINER: VAEGANTrainer
LOGGING: {STEP_VERBOSE: 25}
MODEL: {ARCH: VAEGAN, BASE: 32, LATENT_DIMENSIONS: 256, MODEL_KWARGS: '{"channels":3}'}
OPTIMIZER: {BASE_LR: 0.002, FP16: true, LR_BIAS_FACTOR: 1.0, OPTIMIZER_KWARGS: '{"betas":[0.5,
        0.999]}', OPTIMIZER_NAME: Adam, WEIGHT_BIAS_FACTOR: 0.0005, WEIGHT_DECAY: 0.0005}
SAVE: {DRIVE_BACKUP: true, MODEL_BACKBONE: vaegan, MODEL_CORE_NAME: vaegan_cifar,
    MODEL_QUALIFIER: cifar10, MODEL_VERSION: 4,

Files already downloaded and verified


05:28:45-373 Generated training data generator


Files already downloaded and verified


05:28:46-251 Generated testing data generator
05:28:46-254 Loaded vaegan_model_builder from models to build VAEGAN model
05:28:46-316 Finished instantiating model
05:28:48-738 None
05:28:48-746 None
05:28:48-752 None
05:28:48-765 None
05:28:48-770 Loaded VAEGANOptimizerBuilder from optimizer to build VAEGAN model
05:28:48-772 Built optimizer
05:28:48-776 Built scheduler for Encoder
05:28:48-780 Built scheduler for Decoder
05:28:48-781 Built scheduler for Discriminator
05:28:48-782 Built scheduler for Autoencoder
05:28:48-783 Built scheduler for LatentDiscriminator


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           1,792
            Conv2d-2            [-1, 128, 8, 8]          73,856
       BatchNorm2d-3            [-1, 128, 8, 8]             256
         ConvBlock-4            [-1, 128, 8, 8]               0
            Conv2d-5            [-1, 256, 4, 4]         295,168
       BatchNorm2d-6            [-1, 256, 4, 4]             512
         ConvBlock-7            [-1, 256, 4, 4]               0
            Conv2d-8            [-1, 512, 2, 2]       1,180,160
       BatchNorm2d-9            [-1, 512, 2, 2]           1,024
        ConvBlock-10            [-1, 512, 2, 2]               0
AdaptiveAvgPool2d-11            [-1, 512, 1, 1]               0
           Linear-12                  [-1, 256]         131,328
Total params: 1,684,096
Trainable params: 1,684,096
Non-trainable params: 0
---------------------------

05:28:49-263 Loaded VAEGANTrainer from trainer to build VAEGAN model
05:28:49-272 Starting training
05:28:49-273 Logging to:	vaegan_cifar-v4-vaegan-cifar10-logger.log
05:28:49-274 Models will be saved to local directory:	vaegan_cifar-v4-vaegan-cifar10
05:28:49-276 Models will be backed up to drive directory:	./drive/My Drive/Vehicles/Models/vaegan_cifar-v4-vaegan-cifar10
05:28:49-277 Models will be saved with base name:	vaegan_cifar-v4_epoch[].pth
05:28:49-278 Optimizers will be saved with base name:	vaegan_cifar-v4_epoch[]_optimizer.pth
05:28:49-279 Schedulers will be saved with base name:	vaegan_cifar-v4_epoch[]_scheduler.pth
05:28:49-351 Starting epoch 0 with 230 steps and learning rate 2.00000E-03
05:28:59-623 Epoch0.24	Encoder: 4.191 Decoder: 1.203 AE: 2.902 Discriminator: 1.246 Latent: 1.000
05:29:10-236 Epoch0.49	Encoder: 4.416 Decoder: 3.579 AE: 3.400 Discriminator: 0.172 Latent: 1.218


**bold text**# Encoder test

In [ ]:
#!cp drive/My\ Drive/Vehicles/Models/vaegan_cifar-v4-vaegan-cifar10/*epoch120.pth .

In [ ]:
config="config/vaegan_cifar10.yml"
#weights="vaegan_cifar-v3-vaegan-cifar10/vaegan_cifar-v3_epoch15.pth"
#weights="vaegan_cifar-v4_epoch120.pth"
cfg = kaptan.Kaptan(handler='yaml')
config = cfg.import_config(config)

In [ ]:
# --------------------- INSTANTIATE MODEL ------------------------
model_builder = __import__("models", fromlist=["*"])
model_builder = getattr(model_builder, config.get("EXECUTION.MODEL_BUILDER"))

vaegan_model = model_builder(   arch=config.get("MODEL.ARCH"), base=config.get("MODEL.BASE"), \
                                latent_dimensions = config.get("MODEL.LATENT_DIMENSIONS"), \
                                **json.loads(config.get("MODEL.MODEL_KWARGS")))


In [ ]:
vaegan_model.load_state_dict(torch.load(weights))
vaegan_model.cuda()
vaegan_model.eval()

VAEGAN(
  (Encoder): Encoder(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (extractor): Sequential(
      (0): ConvBlock(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (conv1_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ConvBlock(
        (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (conv1_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): ConvBlock(
        (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (conv1_bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (gap): AdaptiveAvgPool2d(output_size=(1, 1))
    (embedding): Linear(in_features=512, out_features=256, bias=True)
  )
  (Decoder): Decoder(
    (dconv1): ConvTranspose2d(256, 512, kernel_size=(4, 4), strid

In [ ]:
NUM_GPUS = torch.cuda.device_count()
NORMALIZATION_MEAN, NORMALIZATION_STD, RANDOM_ERASE_VALUE = utils.fix_generator_arguments(config)
TRAINDATA_KWARGS = {"rea_value": config.get("TRANSFORMATION.RANDOM_ERASE_VALUE")}

# --------------------- BUILD GENERATORS ------------------------
# Supported integrated data sources --> MNIST, CIFAR
# For BDD or others need a crawler and stuff...but we;ll deal with it later
from generators import ClassedGenerator

load_dataset = config.get("EXECUTION.DATASET_PRELOAD")
if load_dataset in ["MNIST", "CIFAR10", "CIFAR100"]:
    crawler = load_dataset
    #dataset = torchvision.datasets.MNIST(root="./MNIST", train=True,)
else:
    raise NotImplementedError()
    # import crawler
    # crawler = crawler...
    # logger.info("Crawling data folder %s"%config.get("DATASET.ROOT_DATA_FOLDER"))
    # crawler = ReidDataCrawler(data_folder = config.get("DATASET.ROOT_DATA_FOLDER"), train_folder=config.get("DATASET.TRAIN_FOLDER"), test_folder = config.get("DATASET.TEST_FOLDER"), query_folder=config.get("DATASET.QUERY_FOLDER"), **{"logger":logger})

train_generator = ClassedGenerator( gpus=NUM_GPUS, i_shape=config.get("DATASET.SHAPE"), \
                                    normalization_mean=NORMALIZATION_MEAN, normalization_std=NORMALIZATION_STD, normalization_scale=1./config.get("TRANSFORMATION.NORMALIZATION_SCALE"), \
                                    h_flip = config.get("TRANSFORMATION.H_FLIP"), t_crop=config.get("TRANSFORMATION.T_CROP"), rea=config.get("TRANSFORMATION.RANDOM_ERASE"), 
                                    **TRAINDATA_KWARGS)    
train_generator.setup(  crawler, preload_classes = config.get("EXECUTION.DATASET_PRELOAD_CLASS"), \
                        mode='train',batch_size=config.get("TRANSFORMATION.BATCH_SIZE"), \
                        workers = config.get("TRANSFORMATION.WORKERS"))

test_generator = ClassedGenerator( gpus=NUM_GPUS, i_shape=config.get("DATASET.SHAPE"), \
                                    normalization_mean=NORMALIZATION_MEAN, normalization_std=NORMALIZATION_STD, normalization_scale=1./config.get("TRANSFORMATION.NORMALIZATION_SCALE"), \
                                    h_flip = config.get("TRANSFORMATION.H_FLIP"), t_crop=config.get("TRANSFORMATION.T_CROP"), rea=config.get("TRANSFORMATION.RANDOM_ERASE"), 
                                    **TRAINDATA_KWARGS)    
test_generator.setup(  crawler, preload_classes = config.get("EXECUTION.DATASET_TEST_PRELOAD_CLASS"), \
                        mode='test',batch_size=config.get("TRANSFORMATION.BATCH_SIZE"), \
                        workers = config.get("TRANSFORMATION.WORKERS"))

0it [00:00, ?it/s]

 99%|█████████▉| 169615360/170498071 [00:11<00:00, 17005822.24it/s]

Extracting ./CIFAR10/cifar-10-python.tar.gz to ./CIFAR10
Files already downloaded and verified


In [ ]:
new_batch = next(iter(train_generator.dataloader))
batch = new_batch[0].numpy()

In [ ]:
new_test_batch = next(iter(test_generator.dataloader))
test_batch = new_test_batch[0].numpy()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
result = vaegan_model(torch.tensor(batch).cuda()).detach().cpu().numpy()
encoding_disc = vaegan_model.LatentDiscriminator(vaegan_model.Encoder(torch.tensor(batch).cuda()).squeeze()).squeeze()

In [ ]:
test_result = vaegan_model(torch.tensor(test_batch).cuda()).detach().cpu().numpy()
test_encoding_disc = vaegan_model.LatentDiscriminator(vaegan_model.Encoder(torch.tensor(test_batch).cuda()).squeeze()).squeeze()

In [ ]:
encoded_value = vaegan_model.Encoder(torch.tensor(batch).cuda()).squeeze()